# 3.2- Automatización

![automatizacion](images/automatizacion.png)

Selenium automatiza los navegadores. Principalmente, es para la automatización de aplicaciones web con fines de pruebas, pero ciertamente no se limita a eso. Las tareas aburridas de administración basadas en web pueden ser automatizadas.

Veremos como realizar estas automatizaciones y además ocultar nuestro driver para que parezca una persona.

**Librerias**

In [1]:
from selenium import webdriver # driver de selenium

import time   # para manejar tiempo

import pandas as pd

### Opciones del driver

https://peter.sh/experiments/chromium-command-line-switches/


In [2]:
from selenium.webdriver.chrome.options import Options

In [3]:
# opciones del driver

opciones=Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False   # si True, no aparece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

opciones.add_extension('driver/adblock.crx')       # adblocker


opciones.add_argument('user-data-dir=selenium')    # mantiene las coockies


#opciones.add_argument('--incognito')              # incognito
 
#opciones.add_argument('--no-proxy-server')                 # sin proxy
#opciones.add_argument('proxy-server=106.122.8.54:3128')    # proxy custom


### Creando usuario random

https://pypi.org/project/fake-useragent/

In [4]:
%pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.


In [5]:
from fake_useragent import UserAgent

usuario=UserAgent().random

print(usuario)

#opciones.add_argument(f'user-agent={usuario}')

Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.90 Safari/537.36


### Editando el archivo chromedriver


Distil web monitor comprueba el patrón de regex  "[a-z]dc_", y el chromedriver tiene una variable que cumple con dicho patrón "cdc_". Esto hace posible la detección del driver en la red por parte de los monitores.


**Cambiando la string cdc_**

Podemos editar el chromedriver usando Vim, escribiendo en terminal:

`vim /ruta/al/chromedriver`

Una ver abierto el Vim, verás muchas **@** en el archivo. En el editor realizamos los siguientes pasos:

+ Vamos a reemplazar la string `cdc_` todas las veces que aparece escribiendo: `%s/cdc_/arm_/g`. arm_ es solo un ejemplo, se puede sustituir con cualquier string con la condición de que se mantenga la longitud, si no el driver fallará.

+ Para guardar los cambios y salir de Vim se escribe: `:wq!` y presiona Enter. Para salir sin guardar `:q!`.


In [6]:
from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/iudh/.wdm/drivers/chromedriver/mac64_m1/100.0.4896.60/chromedriver] found in cache


### Linkedin sin user

In [7]:
url='https://www.linkedin.com/in/yonatan-rodriguez/'

In [10]:
driver=webdriver.Chrome(PATH, options=opciones)

driver.get(url)

In [11]:
driver.quit()

### Ejemplo Cuevana

https://ww3.cuevana.pro/

In [12]:
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver import ActionChains as AC

In [13]:
url='https://ww3.cuevana.pro/'

In [57]:
driver=webdriver.Chrome(PATH, options=opciones)

driver.get(url)

# busqueda de serie
busqueda=driver.find_element_by_xpath('/html/body/header/div[2]/div/ul/li/a')
busqueda.click()

time.sleep(2)

texto=driver.find_element_by_xpath('/html/body/main/div/input')
texto.send_keys('the boys diabolico')
texto.send_keys(u'\ue007')

time.sleep(2)

# primer resultado
primer_resultado=driver.find_element_by_xpath('/html/body/main/div/div/section[2]/div/div/div/a/div[1]/figure/img')
primer_resultado.click()

# primer capitulo
primer_cap=driver.find_element_by_xpath('/html/body/main/div/article/div/div[2]/ul/li/div[1]/a/img')
primer_cap.click()

time.sleep(2)

# desplegable
dropdown=driver.find_element_by_xpath('//*[@id="player"]/ul/li[2]/div/div[2]')
dropdown.click()


# scroll , codigo javascript
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')


# espera hasta que exista, maximo 3 secs
idioma=WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="player"]/ul/li[2]/ul')))
idioma.click()


# cambiar de frame, de html
iframe=driver.find_element_by_xpath('//*[@id="video-embed"]')

driver.switch_to.frame(iframe)

iframe=driver.find_element_by_xpath('/html/body/iframe[1]')

driver.switch_to.frame(iframe)

time.sleep(2)

# darle al play
play=driver.find_element_by_xpath('//*[@id="vplayer"]/div/div[3]/div[2]')
play.click()


# poner pantalla completa
video=driver.find_element_by_xpath('//*[@id="vplayer"]/div/div[3]/video')

AC(driver).double_click(video).perform()



# manejo tiempo del video

porcentaje=0.

while porcentaje!=100.:
    porcentaje=driver.find_element_by_xpath('//*[@id="vplayer"]/div/div[4]/div[2]/div[1]/div/div[2]').get_attribute('style')
    porcentaje=porcentaje.split()[1].split('%')[0]
    porcentaje=float(porcentaje)
    time.sleep(10)
    
    
# fuera pantalla completa
time.sleep(2)

driver.switch_to.default_content()  # frame original

siguiente=driver.find_element_by_xpath('/html/body/main/ul/li[3]/a')

siguiente.click()

In [56]:
driver.quit()